### Restart and Run All

In [2]:
import calendar
import os
import pandas as pd
import sidetable
import numpy as np
import panel as pn
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from itables import show
pn.extension('tabulator')

# Define a formatting function to format integers with zero decimal places and floats with two decimal places
def format_cell(x):
    if isinstance(x, int):
        return "{:,.0f}".format(x)
    elif isinstance(x, float):
        return "{:.2f}".format(x)
    else:
        return x

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

today = date.today()
print(today)

2025-08-17


### Tables in the process

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Weekly
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [7]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,15,B,AIMIRT,2500,9.8,2,,SET,2022-02-02


In [8]:
names = orders['name']
name = names.to_string(index=False)

In [9]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,AIMIRT,0.2150,0.2150,0.2050,0.2150,0.8500,"17,500",2025-08-19,2025-09-05,,1


In [10]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'AIMIRT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,101,AIMIRT,0.00,0.00,B,9.10,11.50,0.00,0.00,10.90,"2,500",-4,4,"12,500",0.000000,0.000000,RND15pct,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [12]:
sql = text('DELETE FROM orders')
rp = conlite.execute(sql)
rp.rowcount

15

In [13]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy.shape

(12, 7)

In [14]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell.shape

(4, 7)

In [15]:
#df_order = df_buy.append(df_sell, ignore_index=True)
df_order = pd.concat([df_buy, df_sell], ignore_index=True)
df_order.shape

(16, 7)

In [16]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,12,12,12,12,12,12
S,4,4,4,4,4,4


In [17]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(29, 2)

In [18]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(30, 9)

### If there is no dividend record, must create dividend record before proceed

In [20]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
5,B,CPF,4000.0,24.1,0.0,,SET50,NaT,left_only


### Rerun from here

In [22]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,12,12,12,12,12,12,12
S,4,4,4,4,4,4,4


### Restart and Run All Cells

In [24]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,12,12,12,12,12,12,12
S,4,4,4,4,4,4,4


In [25]:
rcds = df_orders.values.tolist()
len(rcds)

16

In [26]:
for rcd in rcds:
    print(rcd)

['B', 'AIMIRT', 2500, 9.1, 0, 'RND15pct', 'SET', '2025-08-19']
['B', 'CPF', 4000, 24.1, 0, '', 'SET50', 'NaT']
['B', 'DIF', 10000, 8.35, 0, 'RD40Pct', 'SET', '2025-08-15']
['B', 'GVREIT', 5000, 6.2, 0, 'RND20Pct', 'SET', '2025-08-27']
['S', 'IVL', 1000, 24.0, 0, 'CSSTGT', 'SET50', '2025-08-27']
['B', 'JMART', 3200, 8.0, 0, 'RD80Pct', 'SET50', '2025-05-09']
['B', 'JMT', 2800, 11.0, 0, 'RD75pct', 'SET50', '2025-08-26']
['S', 'MCS', 6000, 8.75, 0, '20Pct', 'SET', '2025-08-26']
['S', 'NER', 5000, 4.5, 0, 'CSSMIN', 'SET', '2025-08-22']
['S', 'PTG', 3600, 11.4, 0, 'COS', 'SET100', '2025-03-10']
['B', 'PTT', 2500, 31.0, 0, '', 'SET50', '2025-03-06']
['B', 'RCL', 3000, 28.0, 0, 'RD35pct', 'SET100', '2025-08-22']
['B', 'SENA', 15000, 1.44, 0, 'RD70pct', 'SET', '2025-05-09']
['B', 'TFFIF', 10000, 6.0, 0, 'RD30Pct', 'SET', '2025-05-28']
['B', 'WHAIR', 10000, 4.3, 0, 'RD50pct', 'SET', '2025-08-18']
['B', 'WHART', 5000, 8.0, 0, 'RD30Pct', 'SET', '2025-08-18']


In [27]:
sql = text("""
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (:trade, :name, :qty, :price, :active, :reason, :market, :xdate)
""")

for rcd in rcds:
    # Convert list to dictionary with column names
    data = {
        'trade': rcd[0],
        'name': rcd[1],
        'qty': rcd[2],
        'price': rcd[3],
        'active': rcd[4],
        'reason': rcd[5],
        'market': rcd[6],
        'xdate': rcd[7]
    }
    
    conlite.execute(sql, data)
    conlite.commit()    

In [28]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders.shape

(16, 9)

In [29]:
file_name = 'orders.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [30]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\orders.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders.csv


In [31]:
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [34]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,CPF,4000,24.1,0,,SET50,NaT
1,6,B,JMART,3200,8.0,0,RD80Pct,SET50,2025-05-09
2,7,B,JMT,2800,11.0,0,RD75pct,SET50,2025-08-26
3,11,B,PTT,2500,31.0,0,,SET50,2025-03-06
4,5,S,IVL,1000,24.0,0,CSSTGT,SET50,2025-08-27


In [35]:
name = 'BLA'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.000000,20.000000,20.00000,2.000000e+01,20.000000
mean,17.585000,17.845000,17.27000,3.132745e+06,17.490000
std,0.631852,0.704478,0.59921,1.965899e+06,0.658467
min,16.600000,16.800000,16.40000,9.207670e+05,16.600000
25%,17.175000,17.300000,16.70000,1.730924e+06,17.025000
50%,17.550000,17.850000,17.25000,2.604086e+06,17.450000
75%,18.000000,18.350000,17.72500,3.766258e+06,17.925000
max,18.700000,19.300000,18.50000,7.973069e+06,18.700000


### 2) Set100 records

In [37]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,12,B,RCL,3000,28.0,0,RD35pct,SET100,2025-08-22
1,10,S,PTG,3600,11.4,0,COS,SET100,2025-03-10


In [38]:
name = 'KKP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,48.804167,49.287500,48.170833,2.722955e+06,48.612500
std,4.287865,4.184522,4.213701,1.792392e+06,4.158009
min,44.000000,44.750000,43.500000,7.707890e+05,43.750000
25%,45.687500,46.250000,45.000000,1.484853e+06,45.500000
50%,47.125000,47.625000,46.625000,2.224756e+06,47.125000
75%,52.875000,53.375000,51.562500,3.505528e+06,51.812500
max,57.750000,58.000000,57.500000,9.778903e+06,57.750000


### 3) Set records

In [40]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AIMIRT,2500,9.10,0,RND15pct,SET,2025-08-19
1,3,B,DIF,10000,8.35,0,RD40Pct,SET,2025-08-15
2,4,B,GVREIT,5000,6.20,0,RND20Pct,SET,2025-08-27
3,13,B,SENA,15000,1.44,0,RD70pct,SET,2025-05-09
4,14,B,TFFIF,10000,6.00,0,RD30Pct,SET,2025-05-28
5,15,B,WHAIR,10000,4.30,0,RD50pct,SET,2025-08-18
6,16,B,WHART,5000,8.00,0,RD30Pct,SET,2025-08-18
7,8,S,MCS,6000,8.75,0,20Pct,SET,2025-08-26
8,9,S,NER,5000,4.50,0,CSSMIN,SET,2025-08-22


In [41]:
name = 'SCCC'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
formatted_df = df.map(format_cell)

# Display the formatted dataframe
print(formatted_df)

     name        date   price    maxp    minp     qty    opnp
0    SCCC  2025-06-30  142.50  145.50  142.50  54,140  145.50
1    SCCC  2025-06-27  146.00  148.00  145.00  30,507  147.50
2    SCCC  2025-06-26  145.50  146.50  144.00  21,623  144.50
3    SCCC  2025-06-25  144.50  145.00  143.00  21,825  144.00
4    SCCC  2025-06-24  143.50  144.50  143.00  21,774  144.00
..    ...         ...     ...     ...     ...     ...     ...
115  SCCC  2025-01-08  162.00  163.00  162.00  33,332  163.00
116  SCCC  2025-01-07  163.00  164.00  162.00  22,552  162.50
117  SCCC  2025-01-06  162.00  164.00  162.00  65,304  163.00
118  SCCC  2025-01-03  163.00  164.00  163.00   8,681  164.00
119  SCCC  2025-01-02  163.00  164.00  163.00  44,825  163.50

[120 rows x 7 columns]


In [42]:
const.close()
conmy.close()
conlite.commit()
conlite.close()

In [43]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-17 20:17:44
